In [ ]:
# 各セクターごとに全期間にわたって存在する銘柄を特定する
# 各セクターで利用可能な全銘柄を取得
all_stocks_per_sector = {sector: df[df['セクター'] == sector]['銘柄'].unique() for sector in sectors}

# 全期間にわたってデータが存在する銘柄を特定
available_stocks_per_sector = {}
for sector, stocks in all_stocks_per_sector.items():
    available_stocks = []
    for stock in stocks:
        if len(df[(df['セクター'] == sector) & (df['銘柄'] == stock)]) == len(dates):
            available_stocks.append(stock)
    available_stocks_per_sector[sector] = available_stocks

# 各セクターごとに発行額の最大値、中央値、最小値を持つ銘柄を選定
selected_stocks = {}
for sector, stocks in available_stocks_per_sector.items():
    sector_df = df[(df['セクター'] == sector) & (df['銘柄'].isin(stocks))]
    
    max_stock = sector_df.loc[sector_df['発行額'].idxmax()]['銘柄']
    min_stock = sector_df.loc[sector_df['発行額'].idxmin()]['銘柄']
    
    # 中央値に最も近い値を持つ銘柄を特定
    median_val = sector_df['発行額'].median()
    closest_median_stock = (sector_df.iloc[(sector_df['発行額'] - median_val).abs().argsort()[:1]])['銘柄'].values[0]
    
    selected_stocks[sector] = [max_stock, closest_median_stock, min_stock]

# PDFファイルの作成
with PdfPages(pdf_filename) as pdf:
    for sector in sectors:
        for stock in selected_stocks[sector]:
            stock_df = df[(df['セクター'] == sector) & (df['銘柄'] == stock)]
            plt.figure(figsize=(10, 6))
            sns.lineplot(x='Date', y='リターン', data=stock_df)
            plt.title(f'{sector} Sector - {stock} Return Trend')
            plt.xlabel('Date')
            plt.ylabel('Return')
            pdf.savefig()
            plt.close()

